# Project Fletcher

#### Song Lyric Data Gathering


In [12]:
import pandas as pd 
import numpy as np
import os
import re
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from pprint import pprint
import glob
import pickle
import time
import datetime

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [9]:
with open("billboard.pkl", "rb") as f:
    billboard_df = pickle.load(f)

In [253]:
#billboard_df

In [254]:
# billboard_df[billboard_df.Artist == 'Connie Stevens']

# billboard_df[['Song Title', 'Artist']]

# for song,artist in zip(billboard_df['Song Title'], billboard_df['Artist']): 
#     print(artist)

# Using Genius API and BeautifulSoup

In [26]:
total_lyrics_1 = total_lyrics

In [65]:
df_all = billboard_df.merge(lyrics_df, how='left', on=['Song Title', 'Artist'])
df_all.drop_duplicates(inplace=True)
df_all.reset_index(inplace=True)
del df_all['index']
df_all.fillna(value=np.nan, inplace=True)

In [1]:
# BE CAREFUL - TAKES VERY LONG TO RUN 

index = 0 
total_lyrics = {}
for song, artist in zip(no_lyrics['Song Title'], no_lyrics['Artist']): 
    lyrics_dict = {"Song Title":[],
                  "Artist":[],
                  "Lyrics":[]}

    client_access_token = 'AQ6w7HEjzS1jQsmbwGQIK9SIOfUC26up0Z1tMRC5iuMzxTYGAgphmoPT1cPKVowF'
    base_url = 'https://api.genius.com'

    token = 'Bearer {}'.format(client_access_token)
    headers = {'Authorization': token}

    song_title = song
    artist_name = artist 

    lyrics_dict["Song Title"] = song_title
    lyrics_dict["Artist"] = artist_name

    def get_lyrics(song_api_path):
        song_url = base_url + song_api_path
        response = requests.get(song_url, headers=headers)
        json = response.json()
        path = json["response"]["song"]["path"]

        page_url = "https://genius.com" + path
        page = requests.get(page_url)
        print(page_url)
        print(page.status_code)
        soup = BeautifulSoup(page.text, "html5lib")
        [i.extract() for i in soup('script')]
        lyrics = soup.find("div", class_="lyrics").get_text().encode('ascii', 'ignore')
        return lyrics 

    if __name__ == "__main__":
        path = 'search/'
        request_uri = '/'.join([base_url, path])
        params = {'q': song_title}
        time.sleep(2)
        r = requests.get(request_uri, params=params, headers=headers)
        print(r.status_code)
        json = r.json()
        song_info = None
        #print(json)

        #For specifying particular song among list of songs
        #If artist name appears somewhere in artist hit name (catches bands that begin with "The")
        for hit in json["response"]["hits"]:
            if artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():
                #print(hit)
                song_info = hit
                break
        if song_info:
            song_api_path = song_info["result"]["api_path"]
            #print(song_api_path)
            lyrics_dict["Lyrics"] = get_lyrics(song_api_path)
            total_lyrics[index] = lyrics_dict
            index += 1

In [247]:
column_names = ['Song Title', 'Artist', 'Lyrics']
new_lyrics = pd.DataFrame.from_dict(total_lyrics, orient='index')
new_lyrics.columns = column_names

In [249]:
for song, artist, lyrics in zip(new_lyrics['Song Title'], new_lyrics['Artist'], new_lyrics['Lyrics']):     
    df_all.loc[(df_all["Song Title"]==song)&(df_all["Artist"]==artist), "Lyrics"] = lyrics


In [251]:
df_all.isnull().sum()

position         0
Song Title       0
Artist           0
Year             0
Lyrics        2161
dtype: int64

In [262]:
column_names = ['Song Title', 'Artist', 'Lyrics']
new_lyrics = pd.DataFrame.from_dict(total_lyrics, orient='index')
new_lyrics.columns = column_names

In [263]:
for song, artist, lyrics in zip(new_lyrics['Song Title'], new_lyrics['Artist'], new_lyrics['Lyrics']):     
    df_all.loc[(df_all["Song Title"]==song)&(df_all["Artist"]==artist), "Lyrics"] = lyrics

In [269]:
df_all.shape

(6180, 5)

In [268]:
df_all.Lyrics.isnull().sum()

1571

In [273]:
no_lyrics = df_all[df_all.Lyrics.isnull()]

In [364]:
pd.set_option("max_rows", 200)
#no_lyrics

In [2]:

# # BE CAREFUL - TAKES VERY LONG TO RUN 

# index = 0 
# total_lyrics = {}
# for song, artist in zip(no_lyrics['Song Title'], no_lyrics['Artist']): 
#     lyrics_dict = {"Song Title":[],
#                   "Artist":[],
#                   "Lyrics":[]}

#     client_access_token = 'AQ6w7HEjzS1jQsmbwGQIK9SIOfUC26up0Z1tMRC5iuMzxTYGAgphmoPT1cPKVowF'
#     base_url = 'https://api.genius.com'

#     token = 'Bearer {}'.format(client_access_token)
#     headers = {'Authorization': token}

#     song_title = song
#     artist_name = artist

#     lyrics_dict["Song Title"] = song_title
#     lyrics_dict["Artist"] = artist_name

#     def get_lyrics(song_api_path):
#         song_url = base_url + song_api_path
#         response = requests.get(song_url, headers=headers)
#         json = response.json()
#         path = json["response"]["song"]["path"]

#         page_url = "https://genius.com" + path
#         page = requests.get(page_url)
#         print(page_url)
#         print(page.status_code)
#         soup = BeautifulSoup(page.text, "html5lib")
#         [i.extract() for i in soup('script')]
#         lyrics = soup.find("div", class_="lyrics").get_text().encode('ascii', 'ignore')
#         return lyrics 

#     if __name__ == "__main__":
#         path = 'search/'
#         request_uri = '/'.join([base_url, path])
#         params = {'q': song_title,
#                  'per_page': 30,
#                  'page': 20}
#         #time.sleep(2)
#         r = requests.get(request_uri, params=params, headers=headers)
#         print(r.status_code)
#         json = r.json()
#         song_info = None
#         #pprint(json)

#         #For specifying particular song among list of songs 
        
#         for hit in json["response"]["hits"]:
#             if hit["result"]["primary_artist"]["name"].lower() in artist_name.lower() or artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():
#                 #pprint(hit)
#                 song_info = hit
#                 break
#         if song_info:
#             song_api_path = song_info["result"]["api_path"]
#             #pprint(song_api_path)
#             lyrics_dict["Lyrics"] = get_lyrics(song_api_path)
#             total_lyrics[index] = lyrics_dict
#             index += 1

In [339]:
column_names = ['Song Title', 'Artist', 'Lyrics']
new_lyrics = pd.DataFrame.from_dict(total_lyrics, orient='index')
new_lyrics.columns = column_names

for song, artist, lyrics in zip(new_lyrics['Song Title'], new_lyrics['Artist'], new_lyrics['Lyrics']):     
    df_all.loc[(df_all["Song Title"]==song)&(df_all["Artist"]==artist), "Lyrics"] = lyrics
    
no_lyrics = df_all[df_all.Lyrics.isnull()]

In [340]:
no_lyrics.shape

(1544, 5)

In [347]:
import lyricfetcher

In [409]:
no_lyrics.shape

(430, 5)

In [411]:
index = 0 
total_lyrics = {}

for song, artist in zip(no_lyrics['Song Title'], no_lyrics['Artist']): 
    try:
        lyrics = lyricfetcher.get_lyrics('genius', artist, song)
    except:
        lyrics = 404 
    lyrics_dict = {"Song Title":[],
                  "Artist":[],
                  "Lyrics":[]}
    
    lyrics_dict['Lyrics'] = lyrics
    lyrics_dict["Song Title"] = song
    lyrics_dict["Artist"] = artist
    total_lyrics[index] = lyrics_dict
    index += 1
    

In [413]:
# lyrics = lyricfetcher.get_lyrics('genius', 'al hibbler', 'unchained melody')
# lyrics

In [414]:
#pprint(total_lyrics)

In [398]:
# column_names = ['Song Title', 'Artist', 'Lyrics']
# new_lyrics = pd.DataFrame.from_dict(total_lyrics, orient='index')
# new_lyrics.columns = column_names

# for song, artist, lyrics in zip(new_lyrics['Song Title'], new_lyrics['Artist'], new_lyrics['Lyrics']):     
#     df_all.loc[(df_all["Song Title"]==song)&(df_all["Artist"]==artist), "Lyrics"] = lyrics
    
# #no_lyrics = df_all[df_all.Lyrics.isnull()]

In [417]:
df_all.replace(404, np.nan, inplace=True)

# Have 93% of Lyrics

In [419]:
df_all.isnull().sum()

position        0
Song Title      0
Artist          0
Year            0
Lyrics        430
dtype: int64

In [421]:
no_lyrics = df_all[df_all.Lyrics.isnull()]

In [3]:
# dist = no_lyrics.Year.value_counts()

In [4]:
# df_all[df_all.Lyrics.isnull()]

In [456]:
df_all[df_all.Artist == 'John Mayer']

,position,Song Title,Artist,Year,Lyrics
4821,42.0,No Such Thing,John Mayer,2002,"b'\n\n[Verse 1]\n""Welcome to the real world"", ..."
4927,48.0,Your Body Is A Wonderland,John Mayer,2003,"b""\n\n[Verse 1]\nWe got the afternoon\nYou got..."
5162,83.0,Daughters,John Mayer,2005,"b""\n\n[Verse 1]\nI know a girl\nShe puts the c..."
5266,87.0,Waiting on the World to Change,John Mayer,2006,"b""\n\n[Verse 1]\nMe and all my friends\nWe're ..."
5326,47.0,Waiting On The World To Change,John Mayer,2007,"b""\n\n[Verse 1]\nMe and all my friends\nWe're ..."
5439,60.0,Say,John Mayer,2008,Take out of your wasted honor\nEvery little pa...


In [457]:
with open("lyrics_all.pkl", "wb") as f:
    pickle.dump(df_all, f)

In [458]:
#Testing opening pickled model 
with open("lyrics_all.pkl", "rb") as f:
    df_all = pickle.load(f)